## Population growth
* Populations that die are deleted, so they don't grow.
* However all pops that can grow will, regardless of wheter they are starving or not. 
* Starving will lower thier health, which will eventually make them unable to grow. 

Sarving and Death is taken care of elsewhere, so this notebook won't have that filter. 


In [1]:
import sys
import numpy as np
import pandas as pd

# mapping to the modules that make the app
sys.path.insert(0, "..")
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")
sys.path.insert(0, "../../functions/popgrowth/")

import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

#importing the libraries from the app
from app.connectors import cmdb_graph
from app.objects import time
from app.objects import population
from app.functions import language
from helpers import test_queries

c = cmdb_graph.CosmosdbClient()


executing local windows deployment


Much of the Population growth is handled in scripts inside the Azure Function

In [2]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"functions/popgrowth/settings.yml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [3]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at: 2023-08-02T02:24:39.763114+00:00 UTU:28311 >

In [4]:
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 28311}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

In [5]:
params['pop_health_requirement'] = 0.4

In [6]:
# from tools import growth
global_health_manager = population.Global_Pop_Manager(params,c)


In [7]:
global_health_manager.get_pop_health()

pops_df = pd.DataFrame([d['pop'] for d in global_health_manager.data])
species_df = pd.DataFrame([d['species'] for d in global_health_manager.data])
locations_df = pd.DataFrame([d['location'] for d in global_health_manager.data])
factions_df = pd.DataFrame([d['faction'] for d in global_health_manager.data])

In [8]:
# pops_df,species_df,locations_df,factions_df = growth.get_pop_health(c,params)
print(f"lenth of pops == 0 : {len(pops_df)==0}")
pops_df

lenth of pops == 0 : False


,health,isIdle,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,username,objtype,id
0,0.70,false,Obattla Magor,7467985401518,0.421,0.376,0.632,0.338,8822366435138,0.4850,0.4305,0.505,BillmanLocal2,pop,7467985401518
1,0.70,true,Obattla Pe,5100638967625,0.325,0.352,0.606,0.450,8822366435138,0.5280,0.4400,0.459,BillmanLocal2,pop,5100638967625
2,0.70,true,Obattla Van,1800951322768,0.446,0.390,0.549,0.573,8822366435138,0.5610,0.4755,0.423,BillmanLocal2,pop,1800951322768
3,0.70,true,Obattla Rochi,0692394121482,0.452,0.395,0.514,0.546,8822366435138,0.5300,0.4625,0.354,BillmanLocal2,pop,0692394121482
4,0.70,true,Hassia Sin,0248671359952,0.623,0.215,0.704,0.569,4146242421204,0.6365,0.4258,0.662,BillmanLocal2,pop,0248671359952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,0.42,true,Obattla Magorahtur,5911955742831,0.427,0.722,0.578,0.391,8822366435138,0.4845,0.6032,0.197,notebook,pop,5911955742831
170,0.42,true,Hassia Sinkoyma,1684169060276,0.587,0.437,0.313,0.287,4146242421204,0.3000,0.3685,0.610,notebook,pop,1684169060276
171,0.42,true,Ganjtil Wosunven,5698344603003,0.588,0.530,0.667,0.304,4115897493342,0.4855,0.5078,0.540,notebook,pop,5698344603003
172,0.42,true,Zopon Haydorfkerbergganjhra,6894859030208,0.640,0.529,0.366,0.558,9376731217333,0.4620,0.4955,0.630,notebook,pop,6894859030208


In [9]:
species_df

,name,objid,consumes,effuses,viral_resilience,habitat_resilience,username,objtype,id
0,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
1,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
2,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
3,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
4,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
...,...,...,...,...,...,...,...,...,...
169,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
170,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
171,Si,7324266402745,[Organic],"[Organic waste, Plastics]",0.7,0.2,manderson,species,7324266402745
172,Si,7324266402745,[Organic],"[Organic waste, Plastics]",0.7,0.2,manderson,species,7324266402745


In [10]:
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,isHomeworld,username,objtype,id
0,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
1,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
2,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
3,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
4,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
170,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
171,Luraszi,terrestrial,5640544774717,0.861,0.367,1.171,3686964687244,Ne,true,true,true,manderson,planet,5640544774717
172,Luraszi,terrestrial,5640544774717,0.861,0.367,1.171,3686964687244,Ne,true,true,true,manderson,planet,5640544774717


In [11]:
factions_df

,pop_locations,name,objid,lat,long,username,objtype,id,pop_loactions
0,"[[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...",Obattla,8822366435138,-0.149,0.0,BillmanLocal2,faction,8822366435138,NaN
1,"[[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...",Obattla,8822366435138,-0.149,0.0,BillmanLocal2,faction,8822366435138,NaN
2,"[[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...",Obattla,8822366435138,-0.149,0.0,BillmanLocal2,faction,8822366435138,NaN
3,"[[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...",Obattla,8822366435138,-0.149,0.0,BillmanLocal2,faction,8822366435138,NaN
4,"[[0, 0], [-2, 2], [-1, -1], [2, 0], [-1, 3], [...",Hassia,4146242421204,0.149,0.0,BillmanLocal2,faction,4146242421204,NaN
...,...,...,...,...,...,...,...,...,...
169,"[[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...",Obattla,8822366435138,-0.149,0.0,BillmanLocal2,faction,8822366435138,NaN
170,"[[0, 0], [-2, 2], [-1, -1], [2, 0], [-1, 3], [...",Hassia,4146242421204,0.149,0.0,BillmanLocal2,faction,4146242421204,NaN
171,"[[0, 0], [-2, 0], [1, -1], [-1, 1], [2, -2], [...",Ganjtil,4115897493342,-0.121,0.0,manderson,faction,4115897493342,"[[0, 0], [-2, 2], [0, 2], [0, -2], [-2, 0], [2..."
172,"[[0, 0], [0, -2], [2, -2], [-2, -2], [0, 2], [...",Zopon,9376731217333,0.121,0.0,manderson,faction,9376731217333,"[[0, 0], [0, 2], [1, -1]]"


In [12]:
pops_df['roll'] = pops_df['objid'].apply(lambda x: np.random.random())
pops_df['grow'] = pops_df[['wealth','health']].T.mean() >= pops_df['roll']

reproducing_pops = pops_df[pops_df['grow']].drop(['roll','grow'],axis=1)
reproducing_pops

,health,isIdle,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,username,objtype,id
0,0.70,false,Obattla Magor,7467985401518,0.421,0.376,0.632,0.338,8822366435138,0.4850,0.4305,0.505,BillmanLocal2,pop,7467985401518
4,0.70,true,Hassia Sin,0248671359952,0.623,0.215,0.704,0.569,4146242421204,0.6365,0.4258,0.662,BillmanLocal2,pop,0248671359952
5,0.70,true,Obattla Artes,9819392335838,0.488,0.383,0.572,0.570,8822366435138,0.5710,0.4770,0.497,BillmanLocal2,pop,9819392335838
10,0.42,true,Obattla Sertre,3644546786855,0.325,0.441,0.572,0.532,8822366435138,0.3470,0.3470,0.415,BillmanLocal2,pop,3644546786855
13,0.42,true,Hassia Sinfort,4119534664272,0.710,0.179,0.667,0.658,4146242421204,0.3820,0.2990,0.642,BillmanLocal2,pop,4119534664272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,0.70,true,Serzeampro Bilnobha,6726530563660,0.385,0.422,0.372,0.561,4873232607770,0.4665,0.4442,0.377,notebook,pop,6726530563660
165,0.70,true,Ciki Dadcaspark,7561862981860,0.498,0.350,0.476,0.444,0015468195520,0.4600,0.4050,0.383,notebook,pop,7561862981860
166,0.70,true,Ciki Yfu,3929086938020,0.599,0.723,0.570,0.649,0015468195520,0.6095,0.6662,0.502,notebook,pop,3929086938020
169,0.42,true,Obattla Magorahtur,5911955742831,0.427,0.722,0.578,0.391,8822366435138,0.4845,0.6032,0.197,notebook,pop,5911955742831


In [13]:
print(f"{len(reproducing_pops)} of {len(pops_df)} will grow")

64 of 174 will grow


Going to create species objects for the populations that will reproduce

In [14]:
species = [population.species.Species(species_df.drop_duplicates().T.to_dict()[i]) for i in  species_df.drop_duplicates().T.to_dict().keys()]
species

[<species: None; 4291284873310; Koykudra>,
 <species: None; 7324266402745; Si>,
 <species: None; 1245644824657; Towner>,
 <species: None; 5478087080681; Til>]

In [15]:
for i in species:
    global_health_manager.species_dict[i.objid]=i
global_health_manager.species_dict

{'4291284873310': <species: None; 4291284873310; Koykudra>,
 '7324266402745': <species: None; 7324266402745; Si>,
 '1245644824657': <species: None; 1245644824657; Towner>,
 '5478087080681': <species: None; 5478087080681; Til>}

In [16]:
factions = [population.Faction(factions_df.drop_duplicates().T.to_dict()[i]) for i in  factions_df.drop_duplicates().T.to_dict().keys()]
factions

[<faction: None; 8822366435138; Obattla>,
 <faction: None; 4146242421204; Hassia>,
 <faction: None; 4115897493342; Ganjtil>,
 <faction: None; 9376731217333; Zopon>,
 <faction: None; 4232678858631; Tilcasport>,
 <faction: None; 4873232607770; Serzeampro>,
 <faction: None; 0015468195520; Ciki>]

In [17]:
def get_faction(objid):
    res = [i for i in factions if i.objid == objid]
    return res[0]

get_faction(factions[0].objid)

<faction: None; 8822366435138; Obattla>

In [18]:
children = []
events = []
event_edges = []
for i in reproducing_pops.index.to_list():
    s = global_health_manager.species_dict[species_df.loc[i].to_dict()['objid']]
    p = reproducing_pops.loc[i].to_dict()
    #adding the current pop as a defautl to the species.
    s.config['defaults'] = p
    f = factions_df.loc[i].to_dict()
    l = locations_df.loc[i].to_dict()
    child = population.Pop(s)
    child.birthplace = l
    child.inhabitsEdge = {"node1": child.objid, "node2": l['objid'], "label": "inhabits"}
    child.name = p['name']+language.make_word(1).lower()
    get_faction(f['objid']).assign_pop_to_faction(child)
    event = global_health_manager.population_growth_event(p,l,child)
    events.append(event)
    event_edges.append({"node1": event['objid'], "node2": l['objid'], "label": "happenedAt"})
    event_edges.append({"node1": p['objid'], "node2": event['objid'], "label": "caused"})
    children.append(child)    

    


In [19]:
pd.DataFrame(events).head()

,objid,name,label,text,visibleTo,time,username
0,5402522536223,population growth,event,The population (Obattla Magor) inhabiting Dorf...,BillmanLocal2,28311,event
1,9098993397388,population growth,event,The population (Hassia Sin) inhabiting Dorf ha...,BillmanLocal2,28311,event
2,7499488950631,population growth,event,The population (Obattla Artes) inhabiting Dorf...,BillmanLocal2,28311,event
3,6937170054074,population growth,event,The population (Obattla Sertre) inhabiting Dor...,BillmanLocal2,28311,event
4,7913346753034,population growth,event,The population (Hassia Sinfort) inhabiting Dor...,BillmanLocal2,28311,event


In [20]:
children_df = pd.DataFrame([c.get_data() for c in children])
children_df.head()

,name,objid,label,conformity,literacy,aggression,constitution,health,isIn,industry,wealth,factionLoyalty,isIdle
0,Obattla Magorsu,2580389932559,pop,0.587,0.448,0.564,0.493,0.70,8822366435138,0.5285,0.48825,0.538,True
1,Hassia Sinhia,3669027997193,pop,0.542,0.577,0.441,0.455,0.70,4146242421204,0.4480,0.51250,0.582,True
2,Obattla Artesling,3507821778481,pop,0.567,0.626,0.261,0.617,0.70,8822366435138,0.4390,0.53250,0.707,True
3,Obattla Sertregra,3232052404747,pop,0.525,0.532,0.379,0.506,0.42,8822366435138,0.4425,0.48725,0.460,True
4,Hassia Sinfortsul,5302279808710,pop,0.621,0.454,0.485,0.568,0.42,4146242421204,0.5265,0.49025,0.689,True


In [21]:
p

{'health': 0.42,
 'isIdle': 'true',
 'name': 'Zopon Haydorfkerbergganjhra',
 'objid': '6894859030208',
 'conformity': 0.64,
 'literacy': 0.529,
 'aggression': 0.366,
 'constitution': 0.558,
 'isIn': '9376731217333',
 'industry': 0.462,
 'wealth': 0.4955,
 'factionLoyalty': 0.63,
 'username': 'notebook',
 'objtype': 'pop',
 'id': '6894859030208'}

In [22]:
child.get_data()

{'name': 'Zopon Haydorfkerbergganjhrako',
 'objid': '8253886274804',
 'label': 'pop',
 'conformity': 0.609,
 'literacy': 0.399,
 'aggression': 0.51,
 'constitution': 0.548,
 'health': 0.42,
 'isIn': '9376731217333',
 'industry': 0.529,
 'wealth': 0.464,
 'factionLoyalty': 0.578,
 'isIdle': True}

In [30]:
factionedges = []
[f.get_pop_edges(factionedges) for f in factions]
pd.DataFrame(factionedges).head()

,node1,node2,label
0,2580389932559,8822366435138,isIn
1,3507821778481,8822366435138,isIn
2,3232052404747,8822366435138,isIn
3,2752470870305,8822366435138,isIn
4,6345354614006,8822366435138,isIn


In [24]:
childofEdges = [c.childOf for c in children]
pd.DataFrame(childofEdges).head()

,node1,node2,label
0,2580389932559,7467985401518,childOf
1,3669027997193,0248671359952,childOf
2,3507821778481,9819392335838,childOf
3,3232052404747,3644546786855,childOf
4,5302279808710,4119534664272,childOf


In [25]:
isOfEdges = [c.isOfSpecies for c in children]
pd.DataFrame(isOfEdges).head()

,node1,node2,label
0,2580389932559,4291284873310,isOf
1,3669027997193,4291284873310,isOf
2,3507821778481,4291284873310,isOf
3,3232052404747,4291284873310,isOf
4,5302279808710,4291284873310,isOf


In [26]:
inhabitsEdges = [c.inhabitsEdge for c in children]
pd.DataFrame(inhabitsEdges).head()

,node1,node2,label
0,2580389932559,2972908340606,inhabits
1,3669027997193,2972908340606,inhabits
2,3507821778481,2972908340606,inhabits
3,3232052404747,2972908340606,inhabits
4,5302279808710,2972908340606,inhabits


## Validating The Counts of Edges and Nodes Created

In [27]:
assert(len(reproducing_pops) == len(children))
print(f"Expecting {len(reproducing_pops)} children, got {len(children)}")

assert(len(reproducing_pops) == len(isOfEdges))
print(f"Expecting isOfEdges {len(isOfEdges)}")

assert(len(reproducing_pops) == len(inhabitsEdges))
print(f"Expecting inhabitsEdges {len(inhabitsEdges)}")

assert(len(reproducing_pops) == len(childofEdges))
print(f"Expecting childofEdges {len(childofEdges)}")

assert(len(reproducing_pops) == len(factionedges))
print(f"Expecting factionedges {len(factionedges)}")

assert(len(reproducing_pops) == len(events))
print(f"Expecting events {len(events)}")


assert(len(event_edges) == len(events)*2)
print(f"Expecting event_edges {len(event_edges)} (both caused and happenedAt)")


Expecting 64 children, got 64
Expecting isOfEdges 64
Expecting inhabitsEdges 64
Expecting childofEdges 64
Expecting factionedges 64
Expecting events 64
Expecting event_edges 128 (both caused and happenedAt)


## Building the uploaded data

In [28]:
data = {"nodes":[c.get_data() for c in children] + events ,"edges":isOfEdges + inhabitsEdges + childofEdges + factionedges + event_edges }

print(f"The final dataset is {len(data.get('nodes'))} nodes and {len(data.get('edges'))} edges")

c.upload_data('notebook',data)


The final dataset is 128 nodes and 384 edges


It takes a while to upload all of that data. 